# Functions for grabbing data from WRC API

This package contains a range of functions for grabbing and parsing live timing results data from the WRC website via a simple JSON API that is used to generate the official WRC live timing results web pages.


TO DO - consider a scraper class with a requests session embedded in it.

In [45]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [46]:
# Cache results in text notebook
import requests_cache
requests_cache.install_cache('wrc_cache',
                             backend='sqlite',
                             expire_after=300)

In [47]:
# TO DO 
# There is also an: activeSeasonId":19
# IS there something we can get there?

In [ ]:
# TO DO - this should go into a general utils package
def _jsInt(val):
    """Ensure we have a JSON serialisable value for an int.
       The defends against non-JSON-serialisable np.int64."""
    try:
        val = int(val)
    except:
        val = None
        
    return val

In [48]:
#Is this URL constant or picked up relative to each rally?
URL='https://www.wrc.com/ajax.php?contelPageId=176146'

In [57]:
def _getresponse(_url, args, ss={'conn':None}, secondtry=False):
    """Simple function to get response from a post request."""
    
    if ss['conn'] is None or secondtry:
        ss['conn'] = requests.Session()
        try:
            ss['conn'].get('https://www.wrc.com')
        except:
            return None
        
    try:
        r = ss['conn'].post(_url, data=json.dumps(args))
    except: #requests.exceptions.ConnectionError:
        if not secondtry:
            #If there's an error, try once again
            try:
                _getresponse(_url, args, secondtry=True)
            except:
                return None
        else:
            return None
            
    return r

def _get_and_handle_response(_url, args, func, nargs=1, raw=False):
    """Make a request to the API and then return a raw string
       or parse the response with a provided parser function."""
   
    r =  _getresponse(_url,args)
    if raw or not callable(func):
        return r.text
    
    #Make sure we return the desired number of None items in a tuple as a null response
    if not r or r is None or not r.text or r.text=='null':
        return tuple([None for i in range(nargs)])
    
    return func(r)

In [54]:
ACTIVE_RALLY_URL = 'https://www.wrc.com/ajax.php?contelPageId=171091'

In [60]:
def _parseActiveRally(r):
    """Parse active rally response."""
    event = json_normalize(r.json()).drop(columns='eventDays')
    days =  json_normalize(r.json(), 'eventDays').drop(columns='spottChannel.assets')
    channels = json_normalize(r.json(), ['eventDays', 'spottChannel','assets'])
    return (event, days, channels)

def getActiveRally(_url=None, raw=False, func=_parseActiveRally):
    """Get active rally details."""
    
    if not _url:
        _url = ACTIVE_RALLY_URL  
    args= {"command":"getActiveRally","context":None}
    
    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw)


In [61]:
event, days, channels = getActiveRally() #also works with passing URL
display(event.head())
display(days.head())
display(channels.head())
display(event.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.nation.alpha3,winner.nation.ioc,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,BEL,BEL,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762,None


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

In [32]:
#Raw https://webappsdata.wrc.com/srv API?
#Need to create separate package to query that API
#Season info
#_url = 'https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Season%22&maxdepth=1' 
#r = s.get(_url)
#json_normalize(r.json())

In [101]:
CURRENT_SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=181782'

In [102]:
def _parseCurrentSeasonEvents(r):
    """Parse current season events response."""
    current_season_events = json_normalize(r.json(), ['rallyEvents', 'items'], meta='seasonYear').drop(columns='eventDays')
    eventdays = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays']).drop(columns='spottChannel.assets')
    eventchannel = json_normalize(r.json(), ['rallyEvents', 'items', 'eventDays', 'spottChannel','assets'])
    return (current_season_events, eventdays, eventchannel)

def getCurrentSeasonEvents(raw=False, func=_parseCurrentSeasonEvents):
    """Get events for current season"""
    _url=CURRENT_SEASON_URL
    #There seems to be a second UTL giving same data?
    #_url='https://www.wrc.com/ajax.php?contelPageId=183400'
    args = {"command":"getActiveSeason","context":None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw)
        


In [103]:
current_season_events, eventdays, eventchannel = getCurrentSeasonEvents()
display(current_season_events.head())
display(eventdays.head())
display(eventchannel.head())
display(current_season_events.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,False,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


,id,eventDay,spottChannel.id,spottChannel.displayName
0,331,2020-01-22,2,WRC Rallye Monte Carlo
1,334,2020-01-23,2,WRC Rallye Monte Carlo
2,341,2020-01-24,2,WRC Rallye Monte Carlo
3,344,2020-01-25,2,WRC Rallye Monte Carlo
4,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.800x450,content.image.thumbnail,content.image.original,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,227,2020-01-22T15:00:00+00:00,1579705200,2020-01-22T15:40:00+00:00,1579707600,1548,Preview Magazine,None,https://ott.wrc.com/image/480/270/5e1e0da6bd29...,https://ott.wrc.com/image/thumbnail/5e1e0da6bd...,...,https://ott.wrc.com/image/800/450/5e21bfd8e014...,https://ott.wrc.com/image/thumbnail/5e21bfd8e0...,https://ott.wrc.com/image/original/5e21bfd8e01...,2020-01-17 14:30:00.000000,1,+00:00,3,Pay,3,Live
1,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,https://ott.wrc.com/image/original/placeholder...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

## getItinerary

In [62]:
#This seems to work with sdbRallyId=None, returning active rally?

def _parseItinerary(r):
    """Parse itninerary response."""
    itinerary = json_normalize(r.json()).drop(columns='itineraryLegs')
    legs = json_normalize(r.json(),'itineraryLegs' )
    if not legs.empty:
        legs = legs.drop(columns='itinerarySections')
        sections = json_normalize(r.json(),['itineraryLegs', 'itinerarySections'] ).drop(columns=['controls','stages'])
        controls = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'controls' ] )
        stages = json_normalize(r.json(),['itineraryLegs', 'itinerarySections', 'stages' ] )
    else:
        legs = sections = controls = stages = None
    return (itinerary, legs, sections, controls, stages)

def getItinerary(sdbRallyId=None, raw=False, func=_parseItinerary):
    """Get itinerary details for specified rally."""
    args = {"command":"getItinerary",
            "context":{"sdbRallyId":_jsInt(sdbRallyId)}}
    
    return _get_and_handle_response(URL, args, func, nargs=5, raw=raw)


In [63]:
sdbRallyId = 100
itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
display(itinerary.head())
display(legs.head())
display(sections.head())
display(controls.head())
display(stages.head())

,itineraryId,eventId,name,priority
0,240,124,Itinerary,1


,itineraryLegId,itineraryId,startListId,name,legDate,order,status
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed
1,272,240,452,Friday 24th January,2020-01-24,2,Completed
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed


,itinerarySectionId,itineraryLegId,order,name
0,637,273,1,Section 1
1,638,272,2,Section 2
2,639,272,3,Section 3
3,640,275,4,Section 4
4,641,275,5,Section 5


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked
0,6539,124,NaN,TimeControl,TC0,Monaco,Minute,0.00,None,NaN,2020-01-23T16:00:00,2020-01-23T17:00:00+01:00,Completed,All,NoRounding,True
1,6543,124,NaN,TimeControl,TC0A,Tyre Fitting Zone IN,Minute,166.33,02:45:00,9900000.0,2020-01-23T18:45:00,2020-01-23T19:45:00+01:00,Completed,All,NoRounding,True
2,6541,124,NaN,TimeControl,TC0B,Tyre Fitting Zone OUT,Minute,0.35,00:15:00,900000.0,2020-01-23T19:00:00,2020-01-23T20:00:00+01:00,Completed,All,NoRounding,True
3,6593,124,1528.0,TimeControl,TC1,Malijai,Minute,17.08,00:35:00,2100000.0,2020-01-23T19:35:00,2020-01-23T20:35:00+01:00,Completed,All,NoRounding,True
4,6592,124,1528.0,StageStart,SS1,Malijai - Puimichel (Live TV),Minute,17.47,00:03:00,180000.0,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,Interrupted,None,RoundToClosestMinute,True


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5


In [ ]:
def _parseStartlist(r):
    """Parse raw startlist response."""
    startList = json_normalize(r.json()).drop(columns='startListItems')
    startListItems = json_normalize(r.json(), 'startListItems')
    
    return (startList,startListItems)

def getStartlist(startListId, raw=False, func=_parseStartlist):
    """Get a startlist given startlist ID."""
    args={'command': 'getStartlist',
          'context': {'activeItineraryLeg': { 'startListId': startListId} }}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

In [ ]:
startListId = 451
startList,startListItems = getStartlist(startListId)
display(startList.head())
display(startListItems.head())

In [26]:
def _parseCars(r):
    """Parser for raw cars response."""
    cars = json_normalize(r.json()).drop(columns='eventClasses')
    classes = json_normalize(r.json(), 'eventClasses')
    return (cars, classes)

def getCars(sdbRallyId, raw=False, func=_parseCars):
    """Get cars for a specified rally."""
    args = {"command":"getCars","context":{"sdbRallyId":_jsInt(sdbRallyId)}}
    
    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

In [27]:
cars, classes = getCars(sdbRallyId)
display(cars.head())
display(classes.head())
cars.head().columns

,tag,entryId,eventId,driverId,codriverId,manufacturerId,entrantId,groupId,tagId,entryListOrder,...,manufacturer.manufacturerId,manufacturer.name,manufacturer.logoFilename,entrant.entrantId,entrant.name,entrant.logoFilename,group.groupId,group.name,tag.tagId,tag.name
0,NaN,20683,124,524,525,33,166,10,NaN,1,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
1,NaN,20684,124,762,4883,33,166,10,NaN,2,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
2,NaN,20685,124,670,3027,84,91,10,NaN,3,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
3,NaN,20686,124,534,553,84,91,10,NaN,4,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
4,NaN,20687,124,566,2470,26,94,10,NaN,5,...,26,Ford,ford,94,M-SPORT FORD WORLD RALLY TEAM,None,10,WRC,NaN,NaN


,eventClassId,eventId,name
0,582,124,RC1
1,582,124,RC1
2,582,124,RC1
3,582,124,RC1
4,582,124,RC1


Index(['tag', 'entryId', 'eventId', 'driverId', 'codriverId', 'manufacturerId',
       'entrantId', 'groupId', 'tagId', 'entryListOrder', 'identifier',
       'vehicleModel', 'eligibility', 'priority', 'status', 'tyreManufacturer',
       'driver.personId', 'driver.countryId', 'driver.country.countryId',
       'driver.country.name', 'driver.country.iso2', 'driver.country.iso3',
       'driver.firstName', 'driver.lastName', 'driver.abbvName',
       'driver.fullName', 'driver.code', 'codriver.personId',
       'codriver.countryId', 'codriver.country.countryId',
       'codriver.country.name', 'codriver.country.iso2',
       'codriver.country.iso3', 'codriver.firstName', 'codriver.lastName',
       'codriver.abbvName', 'codriver.fullName', 'codriver.code',
       'manufacturer.manufacturerId', 'manufacturer.name',
       'manufacturer.logoFilename', 'entrant.entrantId', 'entrant.name',
       'entrant.logoFilename', 'group.groupId', 'group.name', 'tag.tagId',
       'tag.name'],
      d

In [28]:
def _parseRally(r):
    """Parser for raw rally response."""
    rally = json_normalize(r.json()).drop(columns=['eligibilities','groups'])
    eligibilities = json_normalize(r.json(),'eligibilities')
    groups = json_normalize(r.json(),'groups')
    return (rally, eligibilities, groups)

def getRally(sdbRallyId, raw=False, func=_parseRally):
    """Get rally details for specified rally."""
    args = {"command":"getRally","context":{"sdbRallyId":_jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw)

In [29]:
rally, eligibilities, groups = getRally(sdbRallyId)
display(rally.head())
display(eligibilities.head())
display(groups.head())

,rallyId,eventId,itineraryId,name,isMain,eventClasses
0,153,124,240,WRC,True,None


,0
0,M
1,None
2,WRC2
3,WRC3
4,RGT


,groupId,name
0,10,WRC
1,98,RALLY2
2,9,RGT
3,99,RALLY4
4,6,R3


In [30]:
def _parseOverall(r):
    """Parser for raw overall response."""
    overall = json_normalize(r.json())
    return overall

def getOverall(sdbRallyId, stageId, raw=False, func=_parseOverall):
    """Get overall standings for specificed rally and stage."""
    args = {"command":"getOverall","context":{"sdbRallyId":_jsInt(sdbRallyId),
                                              "activeStage":{"stageId":_jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

In [31]:
stageId = 1528
overall = getOverall(sdbRallyId, stageId)
overall.head()

,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S


In [34]:
def _parseSplitTimes(r):
    """Parser for raw splittimes response."""
    splitPoints = json_normalize(r.json(),'splitPoints')
    entrySplitPointTimes = json_normalize(r.json(), 'entrySplitPointTimes').drop(columns='splitPointTimes')
    splitPointTimes = json_normalize(r.json(), ['entrySplitPointTimes','splitPointTimes'])
    return (splitPoints, entrySplitPointTimes, splitPointTimes)

def getSplitTimes(sdbRallyId,stageId, raw=False, func=_parseSplitTimes):
    """Get split times for specified rally and stage."""
    args = {"command":"getSplitTimes",
            "context":{"sdbRallyId":_jsInt(sdbRallyId),
                       "activeStage":{"stageId":_jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw)

In [35]:
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
display(splitPoints.head())
display(entrySplitPointTimes.head())
display(splitPointTimes.head())

,splitPointId,stageId,number,distance
0,3089,1528,1,3.36
1,3090,1528,2,7.55
2,3091,1528,4,16.40
3,3094,1528,3,15.60


,entryId,startDateTime,startDateTimeLocal,stageTimeDurationMs,stageTimeDuration
0,20683,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,595200,00:09:55.2000000
1,20684,2020-01-23T19:41:00,2020-01-23T20:41:00+01:00,599800,00:09:59.8000000
2,20685,2020-01-23T19:44:00,2020-01-23T20:44:00+01:00,593400,00:09:53.4000000
3,20686,2020-01-23T19:47:00,2020-01-23T20:47:00+01:00,595300,00:09:55.3000000
4,20687,2020-01-23T19:50:00,2020-01-23T20:50:00+01:00,610500,00:10:10.5000000


,splitPointTimeId,splitPointId,entryId,elapsedDurationMs,elapsedDuration,splitDateTime,splitDateTimeLocal
0,101851,3089,20683,95400,PT1M35.4S,2020-01-23T19:39:35.4,2020-01-23T20:39:35.4+01:00
1,101852,3090,20683,227700,PT3M47.7S,2020-01-23T19:41:47.7,2020-01-23T20:41:47.7+01:00
2,101855,3094,20683,532000,PT8M52S,2020-01-23T19:46:52,2020-01-23T20:46:52+01:00
3,101857,3091,20683,554300,PT9M14.3S,2020-01-23T19:47:14.3,2020-01-23T20:47:14.3+01:00
4,101853,3089,20684,95500,PT1M35.5S,2020-01-23T19:42:35.5,2020-01-23T20:42:35.5+01:00


In [49]:
def _parseStageTimes(r):
    """Parser for raw stagetimes response."""
    stagetimes = json_normalize(r.json())
    return stagetimes

def getStageTimes(sdbRallyId,stageId, raw=False, func=_parseStageTimes):
    """Get stage times for specified rally and stage"""
    args = {"command":"getStageTimes",
            "context":{"sdbRallyId":_jsInt(sdbRallyId),
                       "activeStage":{"stageId":_jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [50]:
stagetimes = getStageTimes(sdbRallyId,stageId)
stagetimes.head()

,stageTimeId,stageId,entryId,elapsedDurationMs,elapsedDuration,status,source,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,85682,1528,20685,593400.0,00:09:53.4000000,Completed,Default,1.0,0.0,00:00:00,0.0,00:00:00
1,85717,1528,20683,595200.0,00:09:55.2000000,Completed,Default,2.0,1800.0,00:00:01.8000000,1800.0,00:00:01.8000000
2,85684,1528,20686,595300.0,00:09:55.3000000,Completed,Default,3.0,1900.0,00:00:01.9000000,100.0,00:00:00.1000000
3,85680,1528,20684,599800.0,00:09:59.8000000,Completed,Default,4.0,6400.0,00:00:06.4000000,4500.0,00:00:04.5000000
4,85712,1528,20690,603600.0,00:10:03.6000000,Completed,Default,5.0,10200.0,00:00:10.2000000,3800.0,00:00:03.8000000


In [51]:
def _parseStagewinners(r):
    """Parser for raw stagewinners response."""
    stagewinners = json_normalize(r.json())
    return stagewinners

def getStagewinners(sdbRallyId, raw=False, func=_parseStagewinners):
    """Get stage winners for specified rally."""
    args = {"command":"getStagewinners",
            "context":{"sdbRallyId":_jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [52]:
stagewinners = getStagewinners(sdbRallyId)
stagewinners.head()

,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


Should we return empty dataframes with appropriate columns, or `None`?

An advantage of returning an empty dataframe with labelled columns is that we can also use the column value list as a test of a returned column.

We need to be consistent so we can have a common, consistent way of dealing with empty responses. This means things like `is None` or `pd.DataFrame().empty` both have to be handled.

In [53]:
#COLS_PENALTIES=['penaltyId','controlId','entryId','penaltyDurationMs','penaltyDuration','reason']

def _parsePenalties(r):
    """Parser for raw penalties response."""
    penalties = json_normalize(r.json())
    return penalties

def getPenalties(sdbRallyId, raw=False, func=_parsePenalties):
    """Get penalties for specified rally."""
    args = {"command":"getPenalties",
            "context":{"sdbRallyId":_jsInt(sdbRallyId)}}
    
    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [54]:
penalties = getPenalties(sdbRallyId)
penalties.head()

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE
3,728,6590,20764,50000,PT50S,5 MINS LATE
4,729,6590,20769,10000,PT10S,1 MIN LATE


In [55]:
#COLS_RETIREMENT = ['retirementId','controlId','entryId','reason','retirementDateTime','retirementDateTimeLocal','status']

def _parseRetirements(r):
    """Parser for raw retirements response."""   
    retirements = json_normalize(r.json())
    return retirements

def getRetirements(sdbRallyId, raw=False, func=_parseRetirements):
    """Get retirements for specified rally."""
    args = {"command":"getRetirements",
            "context":{"sdbRallyId":_jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [56]:
retirements = getRetirements(sdbRallyId)
retirements.head()

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent
3,1478,6545,20687,REJOINED,2020-01-24T06:16:00Z,0001-01-01T00:00:00+00:00,Rejoined
4,1479,6545,20710,REJOINED,2020-01-24T06:17:00Z,0001-01-01T00:00:00+00:00,Rejoined


In [57]:
SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=186641'

In [58]:
#How can we look these up?
SEASON_CATEGORIES = {'WRC':"35", "WRC2":"46", "WRC3":"49","JWRC":"58"}

In [61]:
def _parseSeasonCategory(r):
    """Parser for raw season category response."""
    season_category = json_normalize(r.json())
    return season_category

def getSeasonCategory(seasonCategory=SEASON_CATEGORIES['WRC'], raw=False, func=_parseSeasonCategory): 
    """Get championships in season category."""
    args = {"command":"getSeasonCategory",
            "context":{"seasonCategory":seasonCategory}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=1, raw=raw)

In [62]:
getSeasonCategory()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1


In [63]:
SC_COLS = ['id','externalIdDriver','externalIdCoDriver','externalIdManufacturer']

def getChampionshipCodes():
    """Create dataframe of external championship IDs."""
    champs=pd.DataFrame()

    for sc in SEASON_CATEGORIES:
        seasonCategory = SEASON_CATEGORIES[sc]

        champs = champs.append(getSeasonCategory(seasonCategory)[SC_COLS])

    champs.set_index('id', inplace=True)
    champs.rename(columns={'externalIdDriver':'drivers',
                           'externalIdCoDriver':'codrivers',
                           'externalIdManufacturer':'manufacturers'},
                 inplace=True)
    return champs


In [64]:
getChampionshipCodes()

,drivers,codrivers,manufacturers
id,,,
35,37,38,39
46,40,41,43
49,44,45,None
58,46,47,None


In [68]:
def _getChampionshipId(category='WRC', typ='drivers'):
    """Look up external ids for championship by category and championship."""
    champs=getChampionshipCodes()
    championship_activeExternalId = champs.to_dict(orient='index')[int(SEASON_CATEGORIES[category])]
    activeExternalId = championship_activeExternalId[typ]
    return activeExternalId

def _getSeasonId(): 
    event, days, channels = getActiveRallyBase()
    return int(event.loc[0,'season.externalId'])

def _parseChampionship(r):
    """Parser for raw championship response."""
    championship = json_normalize(r.json()).drop(columns=['championshipRounds','championshipEntries'])
    championshipRounds = json_normalize(r.json(), 'championshipRounds' )
    championshipEntries = json_normalize(r.json(), 'championshipEntries')
    return (championship, championshipRounds, championshipEntries)
    
def getChampionship(category='WRC',typ='drivers',
                    season_external_id=None, raw=False, func=_parseChampionship):
    """Get Championship details for specified category and championship.
       If nor season ID is provided, use the external seasonid from the active rally. """
    
    season_external_id = _getSeasonId()
    args = {"command":"getChampionship",
            "context":{"season":{"externalId":season_external_id},
                       "activeExternalId":_getChampionshipId(category,typ)}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=3, raw=raw)

In [69]:
(championship, championshipRounds, championshipEntries) = getChampionship('JWRC', 'drivers')
display(championship)
display(championshipRounds.head())
display(championshipEntries.head())

,championshipId,seasonId,name,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFourDescription,fieldFiveDescription
0,46,6,FIA Junior WRC Championship for Drivers,Drivers,FirstName,LastName,CountryISO3,Manufacturer,TyreManufacturer


,eventId,championshipId,order,event.eventId,event.countryId,event.country.countryId,event.country.name,event.country.iso2,event.country.iso3,event.name,...,event.timeZoneOffset,event.surfaces,event.organiserUrl,event.categories,event.mode,event.trackingEventId,event.clerkOfTheCourse,event.stewards,event.templateFilename,event.locked
0,125,46,1,125,215,215,Sweden,SE,SWE,Rally Sweden,...,60,,,None,Rally,3046,None,None,None,False
1,129,46,2,129,110,110,Italy,IT,ITA,Rally Italia Sardegna,...,60,,,None,Rally,3050,None,None,None,False
2,131,46,3,131,75,75,Finland,FI,FIN,Neste Rally Finland,...,120,,,None,Rally,3052,None,None,None,False
3,134,46,4,134,83,83,Germany,DE,DEU,ADAC Rallye Deutschland,...,60,,,None,Rally,3055,None,None,None,False
4,135,46,5,135,235,235,United Kingdom of Great Britain and Northern I...,GB,GBR,Dayinsure Rally Wales,...,0,,,None,Rally,3056,None,None,None,False


""


In [70]:
getChampionshipCodes().to_dict(orient='index')#[int(SEASON_CATEGORIES['JWRC'])]

{35: {'drivers': 37, 'codrivers': 38, 'manufacturers': 39},
 46: {'drivers': 40, 'codrivers': 41, 'manufacturers': 43},
 49: {'drivers': 44, 'codrivers': 45, 'manufacturers': None},
 58: {'drivers': 46, 'codrivers': 47, 'manufacturers': None}}

In [71]:
def _parseChampionshipStandings(r):
    """Parser for raw champioship standings response."""
    championship_standings = json_normalize(r.json(),'entryResults', meta='championshipId').drop(columns='roundResults')
    round_results = json_normalize(r.json(),['entryResults', 'roundResults'])

    return (championship_standings, round_results)
    
def getChampionshipStandings(category='WRC',typ='drivers',
                             season_external_id=None, raw=False, func=_parseChampionshipStandings ):
    """Get championship standings."""
    season_external_id = _getSeasonId()
    args = {"command":"getChampionshipStandings",
            "context":{
                       "season":{"externalId":season_external_id,
                                 },
                       "activeExternalId":_getChampionshipId(category,typ)}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=2, raw=raw)

In [72]:
championship_standings, round_results = getChampionshipStandings()
display(championship_standings.head())
display(round_results.head())

,championshipEntryId,overallPosition,overallPoints,championshipId
0,751,1,30,37
1,757,2,22,37
2,753,3,17,37
3,758,4,13,37
4,760,5,10,37


,championshipEntryId,championshipId,eventId,position,totalPoints,pointsBreakdown,dropped,status,publishedStatus
0,751,37,124,1,30,25 + 5,False,Finished,Published
1,757,37,124,2,22,18 + 4,False,Finished,Published
2,753,37,124,3,17,15 + 2,False,Finished,Published
3,758,37,124,4,13,12 + 1,False,Finished,Published
4,760,37,124,5,10,10,False,Finished,Published
